<a href="https://colab.research.google.com/github/tomektom10/dataScienceBootCamp/blob/main/sie%C4%87neuronowa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_excel("natural-APR-MAY.xlsx")
#df = pd.read_csv("brent-jn24_dane-5min.csv",sep=";")
df
df.columns
import numpy as np
import calendar
import datetime
import matplotlib.pyplot as plt


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
df

,Timestamp,Open,High,Low,Close,SMA (30),SMA (15)
0,2024-01-04 13:30:00,-0.056,-0.056,-0.058,-0.057,-0.061,-0.059
1,2024-01-04 13:45:00,-0.057,-0.057,-0.058,-0.058,-0.061,-0.059
2,2024-01-04 14:00:00,-0.057,-0.055,-0.057,-0.057,-0.061,-0.059
3,2024-01-04 14:15:00,-0.056,-0.055,-0.057,-0.057,-0.060,-0.059
4,2024-01-04 14:30:00,-0.057,-0.056,-0.058,-0.056,-0.060,-0.058
...,...,...,...,...,...,...,...
375,2024-01-11 14:15:00,-0.070,-0.069,-0.071,-0.070,-0.075,-0.073
376,2024-01-11 14:30:00,-0.070,-0.069,-0.071,-0.070,-0.074,-0.073
377,2024-01-11 14:45:00,-0.069,-0.068,-0.070,-0.069,-0.074,-0.073
378,2024-01-11 15:00:00,-0.069,-0.068,-0.070,-0.069,-0.074,-0.072


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Timestamp  380 non-null    datetime64[ns]
 1   Open       380 non-null    float64       
 2   High       380 non-null    float64       
 3   Low        380 non-null    float64       
 4   Close      380 non-null    float64       
 5   SMA (30)   380 non-null    float64       
 6   SMA (15)   380 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 20.9 KB


In [ ]:
df = df.dropna()

In [ ]:
df["Close"] = df["Close"].str.replace(",", ".").astype(float)

AttributeError: Can only use .str accessor with string values!

In [ ]:
df

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
windows_size = 5
split_point = 0.9
split_point_nb = int(split_point * len(df))

In [ ]:
#skalowanie do sieci nauronowych

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1,1))

In [ ]:
trein,test = scaled_data[0:split_point_nb], scaled_data[split_point_nb:]

In [ ]:
def generuj_okienka(data, windows_size):
  windows = []
  y = []
  for i in range(len(data) -windows_size):
    window = data[i:i+windows_size]
    windows.append(window)
    y.append(data[i+windows_size])
  return np.array(windows),np.array(y)



In [ ]:
x_train, y_train = generuj_okienka(trein, windows_size)

In [ ]:
x_test, y_test = generuj_okienka(test, windows_size)

In [ ]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [ ]:
#ARCHITEKTURA SIECI TU MOGE MODYFOKOWAC moge zwiekszyc LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(windows_size, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

In [ ]:
predictions = model.predict(x_test)


In [ ]:
predictions_real = scaler.inverse_transform(predictions)

In [ ]:
# Plotting the results
plt.figure(figsize=(12, 6))
#plt.plot(train, label='Training Data')
plt.plot(test, label='Actual Test Data')
plt.plot(predictions, label='ARIMA Predictions', color='red')
plt.title('LSTM Predictions vs Actual Data')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
predictions = model.predict(x_test)

In [ ]:
x_test[0]

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mse = mean_squared_error(y_test, predictions)

In [ ]:
mse

In [ ]:
#MSE 10 window = 0.0023883501780096184
 #     15 = 0.0034732914563747044
 # 5 = 0.002798899567247003

In [ ]:
lista_ostatnich_cen = [1.25, 1.26, 1.22, 1.21, 1.15]

In [ ]:
def zrob_predykcje(model, lista_ostatnich_cen, scaler):
  scaled_data_do_przewidywania = scaler.fit_transform(np.array(lista_ostatnich_cen).reshape(-1,1))
  predykcja_ceny = model.predict(np.array([scaled_data_do_przewidywania]))
  predykcja_ceny_real = scaler.inverse_transform(list(predictions[0].reshape(1,-1)))
  return predykcja_ceny_real

In [ ]:
lista_ostatnich_cen = [1.25, 1.26, 1.22, 1.21, 1.15]

In [ ]:
przewidywanie = zrob_predykcje(model, lista_ostatnich_cen, scaler)

In [ ]:
przewidywanie

In [ ]:
scaled_data_do_przewidywania = scaler.fit_transform(np.array(dane_do_przewidywania).reshape(-1,1))

In [ ]:
np.array([scaled_data_do_przewidywania]).shape

In [ ]:
predykcja_ceny = model.predict(np.array([scaled_data_do_przewidywania]))

In [ ]:
predykcja_ceny[0][0]

In [ ]:
predykcja_ceny_real = scaler.inverse_transform(list(predictions[0].reshape(1,-1)))

In [ ]:
predykcja_ceny_real